# 13.3 흉부 엑스선 기반 폐렴 진단 베이스라인 모델
[흉부 엑스선 기반 폐렴 진단 데이터 세트 링크](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia)

## 13.3.1 시드 값 고정 및 GPU 장비 설정

### 시드값 고정

In [1]:
import torch # 파이토치 
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

### GPU 장비 설정

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 13.3.2 데이터 준비

In [3]:
# 데이터 경로
data_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/'

# 훈련, 검증, 테스트 데이터 경로 설정
train_path = data_path + 'train/'
valid_path = data_path + 'val/'
test_path = data_path + 'test/'

### 데이터 증강을 위한 이미지 변환기 정의

In [4]:
from torchvision import transforms

# 훈련 데이터용 변환기
transform_train = transforms.Compose([
                          transforms.Resize((250, 250)),      # 이미지 크기 조정 
                          transforms.CenterCrop(180),         # 중앙 이미지 확대
                          transforms.RandomHorizontalFlip(0.5), # 좌우 대칭
                          transforms.RandomVerticalFlip(0.2), # 상하 대칭
                          transforms.RandomRotation(20),      # 이미지 회전
                          transforms.ToTensor(),              # 텐서 객체로 변환
                          transforms.Normalize((0.485, 0.456, 0.406), 
                                               (0.229, 0.224, 0.225))]) # 정규화

# 테스트 데이터용 변환기
transform_test = transforms.Compose([
                          transforms.Resize((250, 250)),
                          transforms.CenterCrop(180),
                          transforms.ToTensor(),
                          transforms.Normalize((0.485, 0.456, 0.406), 
                                               (0.229, 0.224, 0.225))])

### 데이터셋 및 데이터 로더 생성
 - 데이터셋 클래스를 직접 정의(오버라이딩)할 필요 없음
 - **ImageFolder라는 데이터셋 생성기를 사용**하기 때문
 
  > 타깃값이 같은 이미지끼리 디렉토리로 구분되어 있는경우 활용
   - 정상데이터의 디렉토리(NORMAL)에 담긴 이미지 -> 타깃값 0으로 간주
   - 폐렴데이터의 디렉토리(PNEUMONIA)에 담긴 이미지 -> 타깃값 1로 간주
   
  > ImageFolder 라이브러리는 torchvision.transforms로 만든 변환기만 받도록 설계됨

In [5]:
from torchvision.datasets import ImageFolder

# 훈련 데이터셋
datasets_train = ImageFolder(root=train_path, transform=transform_train)
# 검증 데이터셋
datasets_valid = ImageFolder(root=valid_path, transform=transform_test) 

In [6]:
for image, target in datasets_train: # 훈련 데이터셋은 image, target을 return
    print(image.shape)
    print(target)
    break

torch.Size([3, 180, 180])
0


In [7]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# 제너레이터 시드값 고정
g = torch.Generator()
g.manual_seed(0)

In [8]:
from torch.utils.data import DataLoader

batch_size = 8

loader_train = DataLoader(dataset=datasets_train, batch_size=batch_size, 
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g, num_workers=2)
loader_valid = DataLoader(dataset=datasets_valid, batch_size=batch_size, 
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g, num_workers=2)

## 13.3.3 모델 생성
 - 이미 만들어져 있는 성능좋은 (훈련된)모델을 가져옴

In [9]:
!pip install efficientnet-pytorch==0.7.1

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=a0676f75296361b8f77a1d45af2afe77449da7512a2de60c0fb08716a656216c
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [10]:
from efficientnet_pytorch import EfficientNet
# 모델 생성
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2) # num_classes=2 : 이진분류
# 장비 할당
model = model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b0


In [11]:
len(list(model.parameters())) # model의 weights,bias는 213개 있음

213

In [12]:
for param in model.parameters():
    print(param.shape) # 213개의 weights,bias 중 하나의 shape은 [32, 3, 3, 3]
    print(param.numel()) # numel() : 텐서 객체가 갖는 구성요소의 총 개수 즉, 32*3*3*3
    break

torch.Size([32, 3, 3, 3])
864


In [13]:
[param.numel() for param in model.parameters()]

[864,
 32,
 32,
 288,
 32,
 32,
 256,
 8,
 256,
 32,
 512,
 16,
 16,
 1536,
 96,
 96,
 864,
 96,
 96,
 384,
 4,
 384,
 96,
 2304,
 24,
 24,
 3456,
 144,
 144,
 1296,
 144,
 144,
 864,
 6,
 864,
 144,
 3456,
 24,
 24,
 3456,
 144,
 144,
 3600,
 144,
 144,
 864,
 6,
 864,
 144,
 5760,
 40,
 40,
 9600,
 240,
 240,
 6000,
 240,
 240,
 2400,
 10,
 2400,
 240,
 9600,
 40,
 40,
 9600,
 240,
 240,
 2160,
 240,
 240,
 2400,
 10,
 2400,
 240,
 19200,
 80,
 80,
 38400,
 480,
 480,
 4320,
 480,
 480,
 9600,
 20,
 9600,
 480,
 38400,
 80,
 80,
 38400,
 480,
 480,
 4320,
 480,
 480,
 9600,
 20,
 9600,
 480,
 38400,
 80,
 80,
 38400,
 480,
 480,
 12000,
 480,
 480,
 9600,
 20,
 9600,
 480,
 53760,
 112,
 112,
 75264,
 672,
 672,
 16800,
 672,
 672,
 18816,
 28,
 18816,
 672,
 75264,
 112,
 112,
 75264,
 672,
 672,
 16800,
 672,
 672,
 18816,
 28,
 18816,
 672,
 75264,
 112,
 112,
 75264,
 672,
 672,
 16800,
 672,
 672,
 18816,
 28,
 18816,
 672,
 129024,
 192,
 192,
 221184,
 1152,
 1152,
 28800,
 11

In [14]:
print('모델 파라미터 개수 :', sum(param.numel() for param in model.parameters()))

모델 파라미터 개수 : 4010110


 - 단순 선형 회귀 식 y=ax+b 에서 파라미터는 a, b 두 개
 - efficientnet-b0 모델은 파라미터가 4백만 개 : 딥러닝 모델 훈련시간 오래걸리는 이유

## 13.3.4 모델 훈련 및 성능 검증

### 손실 함수와 옵티마이저 설정

In [15]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss() # 분류문제

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### 훈련 함수 작성

In [17]:
from sklearn.metrics import accuracy_score # 정확도 계산 함수
from sklearn.metrics import recall_score   # 재현율 계산 함수
from sklearn.metrics import f1_score       # F1 점수 계산 함수
from tqdm.notebook import tqdm             # 진행률 표시 막대

def train(model, loader_train, loader_valid, criterion, optimizer, 
          scheduler=None, epochs=2, save_file='model_state_dict.pth'):
    
    valid_loss_min = np.inf # 초기 손실값 초기화(무한대) (검증 데이터용) 

    # 총 에폭만큼 반복
    for epoch in range(epochs):
        print(f'에폭 [{epoch+1}/{epochs}] \n-----------------------------')
        
        # == [ 훈련 ] ==============================================
        model.train()        # 모델을 훈련 상태로 설정
        epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)
        # '반복 횟수'만큼 반복 
        for images, labels in tqdm(loader_train):
            # 이미지, 레이블(타깃값) 데이터 미니배치를 장비에 할당 
            images = images.to(device)
            labels = labels.to(device)
            
            # 옵티마이저 내 기울기 초기화
            optimizer.zero_grad()
            # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
            outputs = model(images)
            # 손실 함수를 활용해 outputs와 labels의 손실값 계산
            loss = criterion(outputs, labels)
            # 현재 배치에서의 손실 추가 (훈련 데이터용)
            epoch_train_loss += loss.item() 
            loss.backward()       # 역전파 수행
            optimizer.step()      # 가중치 갱신
            if scheduler != None: # 스케줄러 학습률 갱신 
                scheduler.step() 

        # 훈련 데이터 손실값 출력
        print(f'\t훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
        
        # == [ 검증 ] ==============================================
        model.eval()         # 모델을 평가 상태로 설정 
        epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용)
        preds_list = []      # 예측값 저장용 리스트 초기화
        true_list = []       # 실젯값 저장용 리스트 초기화
        
        with torch.no_grad(): # 기울기 계산 비활성화
            for images, labels in loader_valid:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_valid_loss += loss.item()
                
                # 예측값 및 실제값 
                preds = torch.max(outputs.cpu(), dim=1)[1].numpy() # torch.max(outputs, dim=1) : 최대값텐서, 최대값의인덱스텐서 return
                true = labels.cpu().numpy() 
    
                preds_list.extend(preds)
                true_list.extend(true)
                
        # 정확도, 재현율, F1 점수 계산
        val_accuracy = accuracy_score(true_list, preds_list)
        val_recall = recall_score(true_list, preds_list)
        val_f1_score = f1_score(true_list, preds_list)

        # 검증 데이터 손실값 및 정확도, 재현율, F1점수 출력
        print(f'\t검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
        print(f'\t정확도 : {val_accuracy:.4f} / 재현율 : {val_recall:.4f} / F1 점수 : {val_f1_score:.4f}')
        # == [ 최적 모델 가중치 찾기 ] ==============================
        # 현 에폭에서의 손실값이 최소 손실값 이하면 모델 가중치 저장 
        if epoch_valid_loss <= valid_loss_min: 
            print(f'\t### 검증 데이터 손실값 감소 ({valid_loss_min:.4f} --> {epoch_valid_loss:.4f}). 모델 저장')
            # 모델 가중치를 파일로 저장 
            torch.save(model.state_dict(), save_file) 
            valid_loss_min = epoch_valid_loss # 최소 손실값 갱신 
    return torch.load(save_file) # 저장한 모델 가중치를 불러와 반환

### 훈련 및 성능 검증

In [18]:
# 모델 훈련
model_state_dict = train(model=model,
                         loader_train=loader_train, 
                         loader_valid=loader_valid,
                         criterion=criterion, 
                         optimizer=optimizer)

에폭 [1/2] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.4597
	검증 데이터 손실값 : 2.7992
	정확도 : 0.5000 / 재현율 : 1.0000 / F1 점수 : 0.6667
	### 검증 데이터 손실값 감소 (inf --> 5.5984). 모델 저장
에폭 [2/2] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.2535
	검증 데이터 손실값 : 2.4548
	정확도 : 0.5625 / 재현율 : 1.0000 / F1 점수 : 0.6957
	### 검증 데이터 손실값 감소 (5.5984 --> 4.9096). 모델 저장


In [19]:
# 최적 가중치 불러오기
model.load_state_dict(model_state_dict)

<All keys matched successfully>

## 13.3.5 예측 및 평가 결과

In [20]:
datasets_test = ImageFolder(root=test_path, transform=transform_test)

loader_test = DataLoader(dataset=datasets_test, batch_size=batch_size, 
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)

### 예측

In [21]:
def predict(model, loader_test, return_true=False):
    model.eval()    # 모델을 평가 상태로 설정
    preds_list = [] # 예측값 저장용 리스트 초기화
    true_list = []  # 실제값 저장용 리스트 초기화

    with torch.no_grad(): # 기울기 계산 비활성
        for images, labels in loader_test:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            
            preds = torch.max(outputs.cpu(), dim=1)[1].numpy() # 예측값
            true = labels.cpu().numpy() # 실제값 

            preds_list.extend(preds)
            true_list.extend(true)

    if return_true:
        return true_list, preds_list
    else:
        return preds_list

In [22]:
true_list, preds_list = predict(model=model, 
                                loader_test=loader_test, 
                                return_true=True)

### 평가 결과

In [23]:
print('#'*5, '최종 예측 결과 평가 점수', '#'*5)
print(f'정확도 : {accuracy_score(true_list, preds_list):.4f}')
print(f'재현율 : {recall_score(true_list, preds_list):.4f}')
print(f'F1 점수 : {f1_score(true_list, preds_list):.4f}')

##### 최종 예측 결과 평가 점수 #####
정확도 : 0.6426
재현율 : 0.9949
F1 점수 : 0.7768
